# Progressive Growing GAN

A generative model in the GAN architecture learns to map points in a latent space to generated images. Latent space only has meaning as it applies to the generative model being trained. Yet, latent space has structure such as interpolation between points and vector arithmetic between points in its space to derive meaningful and targeted effects on generated images.

GANs are effective at generating crisp synthetic images, but are limited in the size of the images that can be generated such 64 × 64 pixels. The **Progressive Growing GAN** is an extension to the GAN that enables training generator models to generate large high-quality images such as photorealistic faces with size 1024 × 1024 pixels.

The key innovation of the Progressive Growing GAN is the incremental increase in the size of images output by the generator. By generating small images at the beginning of training and gradually adding convolutional layers to both the generator and discriminator to produce larger and larger images. For instance, starting with 4 × 4 pixel images, doubling to 8 × 8, 16 × 16, and so on until the desired output resolution is reached. the desired output size is met.

Resource for GAN tutorials:

https://www.tensorflow.org/hub/tutorials

Resource for Progressive Growing GAN:

https://www.tensorflow.org/hub/tutorials/tf_hub_generative_image_module

# Vector Arithmetic in Latent Space

The generator model in the GAN architecture takes a point from the latent space as input and generates a new image. Typically, latent space is a 100-dimensional hypersphere with each variable drawn from a Gaussian distribution with mean of zero and a standard deviation of one. During training, the generator learns how to map points into the latent space with specific output images and this mapping is different each time the model is trained.

The latent space has structure when interpreted by the generator model and this structure can be queried and navigated for a given model. Typically, new images are generated using random points in the latent space. However, latent space can be constructed (e.g. all 0s, all 0.5s, or all 1s) and used as input or a query to generate a specific image. Points in the latent space can be kept and used in simple vector arithmetic to create new points in the latent space, which can be used to generate images.

A series of points can be created on a linear path between two points in the latent space such as two generated images. The series of points can be used to generate a series of images that show a transition between the two generated images.

# Import **tensorflow** library

Import library and alias it:

In [ ]:
import tensorflow as tf

# GPU Hardware Accelerator

To vastly speed up processing, we can use the GPU available from the Google Colab cloud service. Colab provides a free Tesla K80 GPU of about 12 GB. It’s very easy to enable the GPU in a Colab notebook:

1.	click **Runtime** in the top left menu
2.	click **Change runtime** type from the drop-down menu
3.	choose **GPU** from the Hardware accelerator drop-down menu
4.	click **SAVE**

Verify that GPU is available:

In [ ]:
tf.__version__, tf.test.gpu_device_name()

# Install Packages for Creating Animations

We need to install **imageio**, **scikit-image**, and **tensorflow-docs**:

In [ ]:
!pip -q install imageio
!pip -q install scikit-image
!pip install -q git+https://github.com/tensorflow/docs

# Install Libraries

Abseil Python is a logging module implemented on top of standard logging:

In [ ]:
from absl import logging

Install libraries for image processing:

In [ ]:
import imageio
import PIL.Image
import matplotlib.pyplot as plt
from IPython import display
from skimage import transform

The **imageio** package is a Python library that provides an easy interface to read and write a wide range of image data, including animated images, volumetric data, and scientific formats. The **Image** module is used to represent a PIL image. The **plt** module is used for displaying images. The **display** module is a public API for display tools in IPython. The **transform** module is used for image processing.

Import other libraries:

In [ ]:
import numpy as np
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import time

The **hub** module allows access to TensorFlow Hub, which is a repository of trained machine learning models. The **embed** module is used to embed animation in a notebook.

# Generate a Global Random Seed

Set a global random seed:

In [ ]:
tf.random.set_seed(7)

Seed value does not have to be the number we set. Seed values can be set to any number. For reproducibility, use the same seed when comparing experiments!

# Set Dimensions for Latent Spaces

**Latent space** is a compressed representation of a dataset (of observations or events) where similar data points are closer together in space. For instance, two images of dogs are closer together in space than an image of a dog and one of a tree.

Latent space is useful for learning data features and finding simpler representations of data for analysis. As humans, we have an understanding of a broad range of topics and the events belonging to those topics. Latent space aims to provide a similar understanding for a computer model through a quantitative spatial representation.

Compressing a dataset into a latent space helps a model better understand the observed data because the model deals with much smaller variations than it would with the entire dataset. That is, the model deals with a smaller space than it would without compressing the dataset into a latent space.

The terms *high dimensional* and *low dimensional* help us define how specific or how general the kinds of features we want our latent space to learn and represent. High dimensional latent space is sensitive to more specific features of the input data and can sometimes lead to overfitting when there isn't sufficient training data. Low dimensional latent space aims to capture the most important features (or aspects) required to learn and represent the input data.

We set a high dimensional latent space of 512 because the pre-trained model we use for this experiment learned on this latent space:

In [ ]:
latent_dim = 512

The pre-trained model we use maps from a 512-dimensional latent space to images. We can retrieve this value from **module.structured_input_signature** if we don't know beforehand the module we are using.

# Create Functions

## Create a Function to Interpolate Hypershpere

The function finds the space between vectors in space:

In [ ]:
def interpolate_hypersphere(v1, v2, num_steps):
  v1_norm = tf.norm(v1)
  v2_norm = tf.norm(v2)
  v2_normalized = v2 * (v1_norm / v2_norm)
  vectors = []
  for step in range(num_steps):
    interpolated =\
      v1 + (v2_normalized - v1) *\
      step / (num_steps - 1)
    interpolated_norm = tf.norm(interpolated)
    interpolated_normalized =\
      interpolated * (v1_norm / interpolated_norm)
    vectors.append(interpolated_normalized)
  return tf.stack(vectors)

The function initiates latent space interpolation between two randomly initialized vectors. It interpolates between vectors that are non-zero and don't both lie on a line going through the origin and returns the normalized interpolated vectors to the calling environment.

The function begins by creating two Euclidean normed vectors v1 and v2. It then normalizes v2 to have the same norm as v1. It continues by interpolating between the two vectors on the hypersphere (or latent space) to produce a set of vectors based on the number of interpolation steps.

## Create Functions to Display an Image

The first function displays an image using the **PIL** library. The seond function displays an image using *imshow*. from **pyplot**.

In [ ]:
def display_image(image):
  image = tf.constant(image)
  image = tf.image.convert_image_dtype(image, tf.uint8)
  return PIL.Image.fromarray(image.numpy())

def show_image(image):
  plt.imshow(image)
  plt.axis('off')
  plt.show()

##Create a Function to Show Animation

The function uses **imageio** to display animation:

In [ ]:
def animate(images):
  images = np.array(images)
  converted_images = np.clip(
      images * 255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images)
  return embed.embed_file('./animation.gif')

Given a set of images, show an animation.

# Set Verbosity for Error Logging

We just want to see logging errors:

In [ ]:
logging.set_verbosity(logging.ERROR)

# Progressive GAN Experiment

We use the progan-128 pre-trained model to generate realistic celebrity images. The **progan-128** model is a Progressive GAN trained on CelebA for 128x128 images. It maps from a 512-dimensional latent space to images. During training, the latent space vectors are sampled from a normal distribution.

The module takes a tensor (Tensor(tf.float32, shape=[?, 512]) that represents a batch of latent vectors as input and outputs a tensor (Tensor(tf.float32, shape=[?, 128, 128, 3]) that represents a batch of RGB images. The original model is trained on a GPU for 636,801 steps with a batch size 16.

CelebA (CelebFaces Attributes Dataset) is a large-scale face attributes dataset containing more than 200,000 celebrity images. Each image has 40 attribute annotations. Images in this dataset cover large pose variations and background clutter. CelebA also has large diversities, large quantities, and rich annotations including:

* 10,177 number of identities
* 202,599 number of face images
* 5 landmark locations
* 40 binary attributes annotations per image

CelebA can be employed as the training and test sets for the following computer vision tasks: face attribute recognition, face detection, landmark (or facial part) localization, and face editing & synthesis.

## Load the Pre-Trained Model

Load the **progran-128** pre-trained model:

In [ ]:
hub_model = hub.load(
    'https://tfhub.dev/google/progan-128/1')\
    .signatures['default']

To find the progran-128 site, peruse:

https://tfhub.dev/google/progan-128/1

To find all models currently hosted on tfhub.dev that can generate images, peruse:

https://tfhub.dev/s?module-type=image-generator

Get output shapes:

In [ ]:
hub_model.output_shapes

Get dimensions for latent spaces (latent vector):

In [ ]:
hub_model.structured_input_signature

We just verified that CelebA uses 512 dimensional latent space.

## Generate and Display an Image

New images are generated using random points in the latent space. The pre-trained model identifies the closest vector in the latent space and generates an image from that vector.

Create a function to find the closest vector in the latent space:

In [ ]:
def get_module_space_image():
  vector = tf.random.normal([1, latent_dim])
  image = hub_model(vector)['default'][0]
  return image

The function creates a random vector between 1 and 512. It then uses the pretrained model to generate an image.

Display the generated image:

In [ ]:
generated_image = get_module_space_image()
display_image(generated_image)

Not bad! The pre-trained model generates relatively realistic images from the latent space.

## Create a Function to Generate Multiple Images from Latent Space

The function creates two random vectors, interpolates the space between them, and uses the pre-trained model to generate images:

In [ ]:
def interpolate_between_vectors(steps):
  v1 = tf.random.normal([latent_dim])
  v2 = tf.random.normal([latent_dim])
  # creates a tensor with n steps of interpolation between v1 and v2.
  vectors = interpolate_hypersphere(v1, v2, steps)
  # use module to generate images from the latent space.
  interpolated_images = hub_model(vectors)['default']
  return interpolated_images

The function creates two random vectors based on the latent space of 512 dimensions. It then creates a set of vectors from the latent space. It continues by leveraging the trained model on the set of vectors to create a set of new interpolated images.

## Display an Animation

Let's create an animation:

In [ ]:
interpolation_steps = 100
interpolated_images = interpolate_between_vectors(
    interpolation_steps)
animate(interpolated_images)

Pretty amazing! With the help of our pretrained model, we create an animation from two random vectors.

The number of steps makes a big difference in the shaping process! The higher the number of steps, the more interpolated images are created between the random vectors in the latent space. So experiment with the number of steps to see what happens.

## Display Interpolated Image Vectors

Display each interpolated image to monitor the image generation process.

Get the number of interpolated images between random vectors v1 and v2 in the latent space:

In [ ]:
num_imgs = len(interpolated_images)
num_imgs

Show the initial image:

In [ ]:
show_image(interpolated_images[0])

Show the final image:

In [ ]:
show_image(interpolated_images[num_imgs - 1])

So the process begins with the initial image and morphs it into the final image!

Create a function to display generated image vectors from the latent space:

In [ ]:
def generated_images(images, cols, rows):
  columns, rows = cols, rows
  ax = []
  fig = plt.figure(figsize=(20, 20))
  for i in range(columns*rows):
    img = images[i].numpy()
    ax.append(fig.add_subplot(rows, columns, i+1))
    plt.imshow(img)
    plt.axis('off')

Display:

In [ ]:
generated_images(interpolated_images, 10, 10)

Its' fascinating to see how the model is able to morph images as it interpolates vectors from the latent space!

## Interpolate a Vector from an Uploaded Image

We can generate a random vector from the latent space or or upload our own image.

Import requisite library:

In [ ]:
from google.colab import files

Resource:

https://colab.research.google.com/notebooks/io.ipynb

Create function to get an uploaded image:

In [ ]:
def upload_image():
  uploaded = files.upload()
  image = imageio.imread(uploaded[list(uploaded.keys())[0]])
  return transform.resize(image, [128, 128])

Get image from your local drive:

In [ ]:
local_image = upload_image()

Click **Choose Files** to select the image you want to load from your local drive.

Display:

In [ ]:
display_image(local_image)

Create a generated image based on the local image vector:

In [ ]:
vector = tf.dtypes.cast(local_image, tf.float32)
generated_image = hub_model(vector)['default'][0]
display_image(generated_image)

Instead of creating a latent vector, create one from an uploaded image tensor. We do need to convert the local image tensor to a float tensor. Continue by generating an image from the float tensor with the help of progran-128. End by displaying the image. The program-128 model generates a new image based on what it learned from CelebA. So the generated image resembles a human face regardless of what image is uploaded because progran-128 learned on human faces.

# Show Multiple Images

Create a function to return a latent vector and image:

In [ ]:
def get_vector():
  vector = tf.random.normal([1, latent_dim])
  image = hub_model(vector)['default'][0]
  return vector, image

Display images from latent vectors:

In [ ]:
for _ in range(2):
  latent_vector, image = get_vector()
  print (latent_vector[0][0:3])
  show_image(image)

The function returns a vector from the latent space and an is image generated from program-128. Each time the function is executed, images are different because each vector is generated randomly from the latent space, fed into program-128, and generated.

Subplot display:

In [ ]:
rows, cols = 2, 2
plt.figure(figsize=(10, 10))
for i in range(rows*cols):
  plt.subplot(rows, cols, i + 1)
  plt.imshow(get_module_space_image())
  plt.axis('off')

Sometimes images are more realistic than other times.

# Find Closest Latent Vector Experiment

Fix a target image to find the closest latent vector. Use an image generated from the model or upload your own.

For an excellent resource for automating image generation, peruse:

https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/tf_hub_generative_image_module.ipynb

## Set Initial Vector

Generate a seed for reproducibility and set initial vector:

In [ ]:
seed_value = 777
tf.random.set_seed(seed_value)
feature_vector = tf.random.normal([1, latent_dim])

Verify that the feature vector was drawn from the latent space:

In [ ]:
feature_vector.shape

The feature vector is drawn from the 512-dimensional latent space as indicated by shape 1 x 512. So the vector is a 1-dimensional vector with 512 pixel elements.

Change the seed to generate a different image. Be careful to use the same seeds for reproducibility between experiements.

### Display Image from Initial Vector

Display an image from the random vector using the pretrained model:

In [ ]:
display_image(hub_model(feature_vector)['default'][0])

## Create a New Target Image

Let's create a new target image:

In [ ]:
target_image = get_module_space_image()

Verify shape:

In [ ]:
target_image.shape

Display new target image:

In [ ]:
display_image(target_image)

## Create Function to Find Closest Latent Vector

Define a loss function between the target image and the image generated by a latent space variable. Next, use gradient descent to find variable values that minimize the loss:

In [ ]:
def find_closest_latent_vector(
    initial_vector, target_image,
    num_optimization_steps,
    steps_per_image, loss_alg):
  images = []
  losses = []
  vector = tf.Variable(initial_vector)
  optimizer = tf.optimizers.Adam(learning_rate=0.01)
  loss_fn = loss_alg
  for step in range(num_optimization_steps):
    if (step % 100)==0:
      print()
    print('.', end='')
    with tf.GradientTape() as tape:
      image = hub_model(vector.read_value())['default'][0]
      if (step % steps_per_image) == 0:
        images.append(image.numpy())
      target_image_difference = loss_fn(
          image, target_image[:,:,:3])
      # The latent vectors were sampled from a normal distribution. We can get
      # more realistic images if we regularize the length of the latent vector
      # to the average length of vector from this distribution.
      regularizer = tf.abs(tf.norm(vector) - np.sqrt(latent_dim))
      loss = target_image_difference + regularizer
      losses.append(loss.numpy())
    grads = tape.gradient(loss, [vector])
    optimizer.apply_gradients(zip(grads, [vector]))
  return images, losses

The function accepts the initial vector we just created, number of steps, steps per image, and the loss algorithm. It then initializes variables including the initial vector, optimizer, and loss function. The function continues by training for the number of steps. The training loop uses the pre-trained model to generate an image, finds the space between the image we just created and the target image, uses regularization to get more realistic images, calculates loss, applies the gradient descent algorithm, and optimizes the gradients. Once training is completed, the function returns an array of images generated and an array of losses calculated during training.

Create the loss algorithm:

In [ ]:
reduction = tf.keras.losses.Reduction.SUM
mae_loss_algorithm = tf.losses.MeanAbsoluteError(reduction)

Generally, the tf.losses.MeanAbsoluteError API computes the mean of absolute difference between labels and predictions. In our experiment, it computes the mean absolute difference between vectors in latent space and corresponding actual targets.

Clear previous sessions:

In [ ]:
tf.keras.backend.clear_session()

Run the function:

In [ ]:
num_optimization_steps = 200
steps_per_image = 5
mae_images, mae_loss = find_closest_latent_vector(
    feature_vector, target_image, num_optimization_steps,
    steps_per_image, mae_loss_algorithm)

Tweak optimization steps and steps per image to see the impact on the visualization.

## Plot Loss

Plot loss performance:

In [ ]:
plt.plot(mae_loss)
fig = plt.ylim([0, max(plt.ylim())])

Calculate final loss for MAE reduction:

In [ ]:
MAE_loss = mae_loss[num_optimization_steps - 1]
MAE_loss

## Animate Generated Images

Create an animation:

In [ ]:
animate(np.stack(mae_images))

## Compare the Result to the Target

Get number of images generated:

In [ ]:
len(mae_images)

Get image type:

In [ ]:
type(mae_images[0])

Create a function to display images generated from the latent space:

In [ ]:
def closest_latent_images(faces, rows, cols):
  fig = plt.figure(1, (20., 40.))
  for i in range(40):
    plt.subplot(10, 4, i+1)
    plt.imshow(faces[i])
    plt.axis('off')

Display:

In [ ]:
closest_latent_images(mae_images, 10, 4)

During each step of the training loop, the function generates a new image from by leveraging the pre-trained weights from the progan-128. It then compares the new image to the target. Gradually, through gradient descent and loss minimization techniques images become more and more similar to the target. It's magic!

Contrast the first generated image against the target:

In [ ]:
display_image(np.concatenate(
    [mae_images[0], target_image], axis=1))

Show how well the model performs by showing final generated image with target:

In [ ]:
display_image(np.concatenate(
    [mae_images[-1], target_image], axis=1))

Grab 'mae_images\[-1]' to display the final generated image.

Use other display function:

In [ ]:
show_image(np.concatenate(
    [mae_images[-1], target_image], axis=1))

Not bad at all!

## Try a Different Loss Algorithm

Use a MSE instead of MAE:

In [ ]:
reduction = tf.keras.losses.Reduction.SUM
mse_loss_algorithm = tf.losses.MeanSquaredError(reduction)

Clear previous models:

In [ ]:
tf.keras.backend.clear_session()

Generate images from latent space:

In [ ]:
num_optimization_steps = 200
steps_per_image = 5
mse_images, mse_loss = find_closest_latent_vector(
    feature_vector, target_image, num_optimization_steps,
    steps_per_image, mse_loss_algorithm)

Plot loss:

In [ ]:
plt.plot(mse_loss)
fig = plt.ylim([0, max(plt.ylim())])

Calculate final lose for MSE reduction:

In [ ]:
MSE_loss = mse_loss[num_optimization_steps - 1]
MSE_loss

MSE reduction appears to be much better!

Visualize:

In [ ]:
animate(np.stack(mse_images))

Compare final generated image with the actual target image:

In [ ]:
display_image(np.concatenate(
    [mse_images[-1], target_image], axis=1))

Display the MAE comparison:

In [ ]:
display_image(np.concatenate(
    [mae_images[-1], target_image], axis=1))

## Create a Target from an Uploaded Image

Instead of creating a target image with progran-128 from a random vector in the latent space, create a latent vector from an uploaded image and use progran-128 to create a target image from the latent space. 

Create an initial feature vector:

In [ ]:
seed_value = 0
tf.random.set_seed(seed_value)
feature_vector = tf.random.normal([1, latent_dim])

Grab an image from a local drive and display:

In [ ]:
uploaded_image = upload_image()
display_image(uploaded_image)

Get shape:

In [ ]:
uploaded_image.shape

Convert uploaded image to float32:

In [ ]:
uploaded_vector = tf.dtypes.cast(uploaded_image, tf.float32)
display_image(uploaded_vector)

The vector is not the target image because it was not drawn from the latent space. Use progran-128 to generate the target image from a vector in the latent space:

In [ ]:
uploaded_target = hub_model(uploaded_vector)['default'][0]
display_image(uploaded_target)

Use progran-128 to generate a new target image from the latent space and display it.

Create a loss algorithm with MSE reduction:

In [ ]:
reduction = tf.keras.losses.Reduction.SUM
loss_algorithm = tf.losses.MeanSquaredError(reduction)

Clear previous model sessions:

In [ ]:
tf.keras.backend.clear_session()

Train:

In [ ]:
num_optimization_steps = 300
steps_per_image = 5
mse_images, mse_loss = find_closest_latent_vector(
    feature_vector, uploaded_target, num_optimization_steps,
    steps_per_image, loss_algorithm)

We trained the model on more optimization steps to generate a better facimile of the target image.

Animate:

In [ ]:
animate(np.stack(mse_images))

Compare final generated image to the target:

In [ ]:
display_image(np.concatenate(
    [mse_images[-1], uploaded_target], axis=1))

Not bad. We can increase the number of optimization steps to generate a more realistic image. But be careful because setting the number of steps too high might compromise available RAM.

## Create a Target from a Google Drive Image

Instead of grabbing an image from a local drive, grab it from Google Drive. We create a new feature vector, but you can just use the one created for the uploaded image exercise.

Create an initial feature vector:

In [ ]:
seed_value = 0
tf.random.set_seed(seed_value)
feature_vector = tf.random.normal([1, latent_dim])

Mount Google Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Click on the URL, choose a Gmail account, copy the authorization code, paste it into the text box, and click the **Enter** button on your keypad.

Get the image:

In [ ]:
from PIL import Image

p1 = 'gdrive/My Drive/Colab Notebooks/'
p2 = 'images/honest_abe.jpeg'
path = p1 + p2
img_path = path
gdrive_image = Image.open(img_path)
plt.axis('off')
_ = plt.imshow(gdrive_image)

Create a path to the image on Google Drive. Open the image with the path and display. Be sure that the image is in the **Colab Notebooks** directory!

Reformat the PIL image to the expected size of program-128:

In [ ]:
def reformat(img, size):
  img = tf.keras.preprocessing.image.img_to_array(img) / 255.
  img = tf.image.resize(img, size)
  return img

img_size = (128, 128)
gdrive_vector = reformat(gdrive_image, img_size)
gdrive_vector.shape

The function converts the PIL image to a NumPy array and resizes it to the expected size of progran-128 (128 x 128 x 3).

Display:

In [ ]:
display_image(gdrive_vector)

The array is not the target image because it was not drawn from the latent space. Use progran-128 to generate the target image from a vector in the latent space:

In [ ]:
gdrive_target = hub_model(gdrive_vector)['default'][0]
display_image(gdrive_target)

Create a loss algorithm with MSE reduction:

In [ ]:
reduction = tf.keras.losses.Reduction.SUM
loss_algorithm = tf.losses.MeanSquaredError(reduction)

Clear previous model sessions:

In [ ]:
tf.keras.backend.clear_session()

Train:

In [ ]:
num_optimization_steps = 300
steps_per_image = 5
mse_images, mse_loss = find_closest_latent_vector(
    feature_vector, gdrive_target, num_optimization_steps,
    steps_per_image, loss_algorithm)

Animate:

In [ ]:
animate(np.stack(mse_images))

Compare final generated image to target:  

In [ ]:
display_image(np.concatenate(
    [mse_images[-1], gdrive_target], axis=1))

## Create a Target from Wikimedia Commons

Grab an image from Wikimedia Commons.

Generate a seed and create a feature vector:

In [ ]:
seed_value = 0
tf.random.set_seed(seed_value)
feature_vector = tf.random.normal([1, latent_dim])

Get the image:

In [ ]:
p1 = 'http://upload.wikimedia.org/wikipedia/commons/'
p2 = 'd/de/Wikipedia_Logo_1.0.png'
URL = p1 + p2
im = imageio.imread(URL)
im.shape

Convert the image to a NumPy array and display:

In [ ]:
from keras.preprocessing.image import img_to_array

img_array = img_to_array(im)
print(img_array.dtype)
print(img_array.shape)
plt.imshow(tf.squeeze(img_array))
fig = plt.axis('off')

Resize image for progran-128 consumption:

In [ ]:
wiki_vector = tf.image.resize(img_array, (128, 128))
plt.imshow(tf.squeeze(wiki_vector))
fig = plt.axis('off')

Create the target:

In [ ]:
wiki_target = hub_model(wiki_vector)['default'][0]
display_image(wiki_target)

Create a loss algorithm with MSE reduction:

In [ ]:
reduction = tf.keras.losses.Reduction.SUM
loss_algorithm = tf.losses.MeanSquaredError(reduction)

Clear previous model session:

In [ ]:
tf.keras.backend.clear_session()

Train:

In [ ]:
num_optimization_steps = 300
steps_per_image = 5
mse_images, mse_loss = find_closest_latent_vector(
    feature_vector, wiki_target, num_optimization_steps,
    steps_per_image, loss_algorithm)

Animate:

In [ ]:
animate(np.stack(mse_images))

Compare:

In [ ]:
display_image(np.concatenate(
    [mse_images[-1], wiki_target], axis=1))

# Latent Vectors and Image Arrays

The progran-128 module generates a new image from either a latent vector of size (1, 512) or float vector of size 128 x 128. A latent vector accepted by progran-128 is a 1-dimensional vector of size 512. A float vector accepted by progran-128 is 128 x 128 pixel vector.

The proran-128 module is an image generator based on the TensorFlow re-implementation of Progressive GANs. It maps from a 512-dimensional latent space to images. During training, latent space vectors are sampled from a normal distribution.

According to the documentation, progran-128 takes an input tensor with datatype float32 tensor and shape (?, 512). The input tensor to progran-128 represents a batch of latent vectors. The output from progran-128 is a float tensor with shape (?, 128, 128, 3), which represents a batch of RGB images. We can also generate a new image from an image array, which is not included in the documentation.

To view progran-128 documentation, peruse:

https://tfhub.dev/google/progan-128/1

## Generate a New Image from a Latent Vector

Create a random normal vector from the latent space:

In [ ]:
random_normal_latent_vector = tf.random.normal([1, latent_dim])
random_normal_latent_vector.shape

The tf.random.normal API outputs random values from a normal distribution. So the new vector consists of 512 randomly drawn values from a normal distribution of the latent space.

Convert the tensor to NumPy to enable inspection:

In [ ]:
rnlv = random_normal_latent_vector.numpy()
len(rnlv[0])

Inspect some elements:

In [ ]:
for i, element in enumerate(rnlv[0]):
  if i < 5:
    print (element)
  else: break

Each element in the new vector represents a latent dimension (or latent variable) that cannot be directory observed, but can be assumed to exist. Since latent dimensions exist, they can be used to explain patterns of variation in observed variables. In our experiment, observed variables represent CelebA images. So we can feed program-128 the new vector to generate a new image.

Create a float output tensor from the latent space with progran-128:

In [ ]:
float_output_tensor = hub_model(
    random_normal_latent_vector)['default'][0]
float_output_tensor.shape

Display the float output tensor as an image:

In [ ]:
display_image(generated_image)

So progran-128 generates a 128 x 128 x 3 image from a latent vector.

## Generate a New Image from an Image Vector 

Get an image from Google Drive:

In [ ]:
p1 = 'gdrive/My Drive/Colab Notebooks/'
p2 = 'images/honest_abe.jpeg'
path = p1 + p2
img_path = path
abe_image = Image.open(img_path)
plt.axis('off')
_ = plt.imshow(abe_image)

Convert the JPEG image to an image vector of the appropriate datatype and size:

In [ ]:
img_size = (128, 128)
abe_vector = reformat(abe_image, img_size)
abe_vector.shape

Display a slice from the vector:

In [ ]:
abe_vector[0][0].numpy()

Generate a new image from the abe vector:

In [ ]:
image_from_abe_vector = hub_model(abe_vector)['default'][0]
display_image(image_from_abe_vector)